# Importing the data

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import bambi, os, glob
import statsmodels as sm
import statsmodels.formula.api as smf

In [6]:
dataDir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data/usable/extracted'))
print dataDir
allSubjFilePaths = glob.glob(dataDir + os.sep + 'FC_cfs*clean.csv')
print os.path.basename(allSubjFilePaths[0])
subjNum = len(allSubjFilePaths)
print 'number of subjects: ' + str(subjNum)
df = pd.DataFrame()
for curSubjFileNum in range(subjNum):
    print 'current subject file number = ' + str(curSubjFileNum)
    ds = pd.read_csv(allSubjFilePaths[curSubjFileNum])
    ds.columns = ['subjId', 'domEyeR', 'threshStHi', 'threshStLo', 'thresh', 'trialN',
                  'sentId', 'sentPx', 'congr', 'fam', 'locTop', 'cued', 'crct', 'broken', 'st']
    df = df.append(ds)
print df.shape
df.head(5)

/Users/Egor/Dropbox/Projects/fc/fc/data/usable/extracted
FC_cfs_sub01_20160915_1402_clean.csv
number of subjects: 10
current subject file number = 0
current subject file number = 1
current subject file number = 2
current subject file number = 3
current subject file number = 4
current subject file number = 5
current subject file number = 6
current subject file number = 7
current subject file number = 8
current subject file number = 9
(2400, 15)


,subjId,domEyeR,threshStHi,threshStLo,thresh,trialN,sentId,sentPx,congr,fam,locTop,cued,crct,broken,st
0,1,1,0.24862,0.2398,0.24421,1,10,124,1,0,0,0,1,1,1.1426
1,1,1,0.24862,0.2398,0.24421,2,15,121,0,1,1,1,1,1,1.7793
2,1,1,0.24862,0.2398,0.24421,3,28,100,0,0,0,0,1,1,1.3451
3,1,1,0.24862,0.2398,0.24421,4,28,100,0,0,1,1,1,1,1.4134
4,1,1,0.24862,0.2398,0.24421,5,2,133,0,1,0,0,1,1,1.5021


## Centered data set

In [7]:
dfc = df
dfc.ix[dfc['cued']==0,'cued'] = -1
dfc.ix[dfc['fam']==0,'fam'] = -1
dfc.ix[dfc['congr']==0,'congr'] = -1

# Statistical models

## Mixed linear regression

In [13]:
md = smf.mixedlm('st ~ congr * fam * cued', dfc, re_formula='sentId', groups=dfc['subjId'])
mdf = md.fit()
print(mdf.summary())

               Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    st        
No. Observations:      2400       Method:                REML      
No. Groups:            10         Scale:                 0.3840    
Min. group size:       240        Likelihood:            -2354.2633
Max. group size:       240        Converged:             Yes       
Mean group size:       240.0                                       
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                 1.549    0.196  7.897 0.000  1.165  1.934
congr                     0.009    0.013  0.689 0.491 -0.016  0.033
fam                       0.020    0.013  1.553 0.121 -0.005  0.044
congr:fam                 0.013    0.013  1.003 0.316 -0.012  0.037
cued                     -0.015    0.012 -1.235 0.217 -0.039  0

/Library/Python/2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


## Running the Bayesian mixed model

In [15]:
bmd = bambi.Model(df)
modelFitted = bmd.fit('st ~ congr * fam * cued', samples=200,
                        random=['1|subjId'], njobs=1)
modelFitted.plot(burn_in=50)

Applied log-transform to u_subjId_sd and added transformed u_subjId_sd_log_ to model.
INFO:pymc3:Applied log-transform to u_subjId_sd and added transformed u_subjId_sd_log_ to model.
Applied interval-transform to st_sd and added transformed st_sd_interval_ to model.
INFO:pymc3:Applied interval-transform to st_sd and added transformed st_sd_interval_ to model.
Assigned NUTS to b_Intercept
INFO:pymc3:Assigned NUTS to b_Intercept
Assigned NUTS to b_congr
INFO:pymc3:Assigned NUTS to b_congr
Assigned NUTS to b_fam
INFO:pymc3:Assigned NUTS to b_fam
Assigned NUTS to b_congr:fam
INFO:pymc3:Assigned NUTS to b_congr:fam
Assigned NUTS to b_cued
INFO:pymc3:Assigned NUTS to b_cued
Assigned NUTS to b_congr:cued
INFO:pymc3:Assigned NUTS to b_congr:cued
Assigned NUTS to b_fam:cued
INFO:pymc3:Assigned NUTS to b_fam:cued
Assigned NUTS to b_congr:fam:cued
INFO:pymc3:Assigned NUTS to b_congr:fam:cued
Assigned NUTS to u_subjId_sd_log_
INFO:pymc3:Assigned NUTS to u_subjId_sd_log_
Assigned NUTS to u_subjId
I

PositiveDefiniteError: Scaling is not positive definite. Simple check failed. Diagonal contains negatives. Check indexes [11 19]